In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import classification_report

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
import tensorflow_model_optimization as tfmot

from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
import pickle
import math
import seaborn
import matplotlib.pyplot as plt

In [3]:
def cleanUp(df):
    df.drop(columns=["Team_abbrev", "Opponent_abbrev", "vis_team", "home_team", "Roof", "Surface"], inplace = True)

    df.drop(columns=["Vegas_Favorite", "total_ret_td", "offensive_fumble_recovery_td"], inplace=True)

    df.drop(columns=["game_date"], inplace=True)

    df.drop(columns=["Vegas_Line", "Over_Under"], inplace=True)

In [4]:
def normalize_data(df):
    df = (df - df.mean())/df.std()
    df.dropna(axis=1, how='all', inplace=True)
    return df

In [5]:
def split_data(df, string):
    Y = np.asarray(df[string])
    df.drop(columns = [string], inplace = True)
    X = np.asarray(df).astype(np.float32)
    X = X.reshape(X.shape[0], -1)
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=.2)
    return X,Y, Xtrain, Xtest, ytrain, ytest

In [6]:
def train(Xtrain, ytrain):
    model_forest = RandomForestRegressor()
    forest=model_forest.fit(Xtrain, ytrain)
    return forest

In [7]:
def analyze(df, string,model, Xtest, ytest):
    ypred = model.predict(Xtest)
    mn = df[string].mean()
    st = df[string].std()
    ypred  = (ypred * st) + mn
    ytest = (ytest * st) + mn
    ypred = np.around(ypred)
    ytest = np.around(ytest)
    return ypred, ytest

In [8]:
def save_model(model, name):
    filename = 'model' + name + '.sav'
    pickle.dump(model, open(filename, 'wb'))

In [9]:
def create_models(lst):
    df_nfl_players_stats = pd.read_csv("C:/Users/carlo/Downloads/nfl_pass_rush_receive_raw_data.csv")
    df_nfl_players_stats['index']= range(1, len(df_nfl_players_stats) + 1)
    current_stadistic = lst
    cleanUp(df_nfl_players_stats)
    norm = normalize_data(df_nfl_players_stats)
    X,Y, Xtrain, Xtest, ytrain, ytest = split_data(norm, current_stadistic)
    model = train(Xtrain, ytrain)
    ypred, ytest = analyze(df_nfl_players_stats, current_stadistic, model, Xtest, ytest)
    save_model(model, current_stadistic) 

In [10]:
df_nfl_players_stats = pd.read_csv("C:/Users/carlo/Downloads/nfl_pass_rush_receive_raw_data.csv")

C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
df_nfl_players_stats.reset_index(inplace=True)

In [12]:
df_nfl_players_stats = pd.read_csv("C:/Users/carlo/Downloads/nfl_pass_rush_receive_raw_data.csv")
df_nfl_players_stats['index']= range(1, len(df_nfl_players_stats) + 1)
cleanUp(df_nfl_players_stats)
norm = normalize_data(df_nfl_players_stats)

In [13]:
overall = df_nfl_players_stats.groupby(["player_id", "game_id"]).mean()
overall.reset_index(inplace=True)

In [14]:
weekly = df_nfl_players_stats.groupby(["player_id"]).last()

In [15]:
weekly

,game_id,pos,player,team,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,...,rec_drops,offense,off_pct,vis_score,home_score,OT,Temperature,Humidity,Wind_Speed,index
player_id,,,,,,,,,,,,,,,,,,,,,
AbduAm00,202301070rai,RB,Ameer Abdullah,LVR,0,0,0,0,0,0,...,0,18,26,31,13,False,72,45,0,26244
AdamDa01,202301070rai,WR,Davante Adams,LVR,0,0,0,0,0,0,...,1,63,90,31,13,False,72,45,0,26239
AdamJo03,202101030nwe,RB,Josh Adams,NYJ,0,0,0,0,0,0,...,0,24,38,14,28,False,33,75,5,12529
AdamRo01,202112050chi,WR,Rodney Adams,CHI,0,0,0,0,0,0,...,1,10,13,33,22,False,34,80,8,17481
AghoNe00,202301080buf,WR,Nelson Agholor,NWE,0,0,0,0,0,0,...,0,4,6,23,35,False,30,74,6,26289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZappBa00,202210240nwe,QB,Bailey Zappe,NWE,14,22,185,1,2,1,...,0,34,65,33,14,False,72,45,0,22578
ZennZa00,201910310crd,RB,Zach Zenner,ARI,0,0,0,0,0,0,...,0,1,2,28,25,False,72,45,0,2873
ZubeIs00,202011010buf,WR,Isaiah Zuber,NWE,0,0,0,0,0,0,...,0,6,9,21,24,False,41,87,17,8971


In [16]:
weekly.reset_index(inplace=True)

In [17]:
weekly

,player_id,game_id,pos,player,team,pass_cmp,pass_att,pass_yds,pass_td,pass_int,...,rec_drops,offense,off_pct,vis_score,home_score,OT,Temperature,Humidity,Wind_Speed,index
0,AbduAm00,202301070rai,RB,Ameer Abdullah,LVR,0,0,0,0,0,...,0,18,26,31,13,False,72,45,0,26244
1,AdamDa01,202301070rai,WR,Davante Adams,LVR,0,0,0,0,0,...,1,63,90,31,13,False,72,45,0,26239
2,AdamJo03,202101030nwe,RB,Josh Adams,NYJ,0,0,0,0,0,...,0,24,38,14,28,False,33,75,5,12529
3,AdamRo01,202112050chi,WR,Rodney Adams,CHI,0,0,0,0,0,...,1,10,13,33,22,False,34,80,8,17481
4,AghoNe00,202301080buf,WR,Nelson Agholor,NWE,0,0,0,0,0,...,0,4,6,23,35,False,30,74,6,26289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,ZappBa00,202210240nwe,QB,Bailey Zappe,NWE,14,22,185,1,2,...,0,34,65,33,14,False,72,45,0,22578
1152,ZennZa00,201910310crd,RB,Zach Zenner,ARI,0,0,0,0,0,...,0,1,2,28,25,False,72,45,0,2873
1153,ZubeIs00,202011010buf,WR,Isaiah Zuber,NWE,0,0,0,0,0,...,0,6,9,21,24,False,41,87,17,8971
1154,ZylsBr00,202201090tam,WR,Brandon Zylstra,CAR,0,1,0,0,0,...,0,50,69,17,41,False,80,99,4,19650


In [53]:
current_stadistic = "pass_att"
weekly['index']= range(1, len(weekly) + 1)
norm = normalize_data(weekly)
X,Y, Xtrain, Xtest, ytrain, ytest = split_data(norm, current_stadistic)

In [19]:
stats_to_be = ["pass_att", "pass_cmp","pass_yds", "pass_td","rush_td", "rush_att", "rush_yds", "targets", "rec", "rec_td", "rec_yds"]

In [20]:
for i in stats_to_be:
    create_models(i)

C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\carlo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\carlo\anaconda3\lib\site-packages\IPython\c

In [33]:
import joblib
filename = "modelpass_att.sav"
loaded_model = joblib.load(filename)

In [54]:
ypred = loaded_model.predict(X)

In [57]:
ypred, ytest = analyze(df_nfl_players_stats, current_stadistic, loaded_model, X, Y)

In [74]:
norm[0]

KeyError: 0

In [72]:
norm.iloc[577]

Humidity                  -0.899943
OT                        -0.243981
Off_DKP                   -0.224909
Off_FDP                   -0.232514
Off_SDP                   -0.246421
Temperature               -1.936356
Total_DKP                 -0.224909
Total_FDP                 -0.232514
Total_SDP                 -0.246421
Wind_Speed                -0.546736
comb_pass_play            -0.277864
comb_pass_rush_play       -0.406847
comb_rush_play            -0.399557
designed_rush_att         -0.399557
fumbles_lost              -0.190054
home_score                -0.596038
index                     -0.001498
off_pct                   -0.897340
offense                   -0.900818
pass_blitzed              -0.253438
pass_cmp                  -0.272693
pass_hurried              -0.193648
pass_int                  -0.205827
pass_long                 -0.276675
pass_poor_throws          -0.247690
pass_rating               -0.321819
pass_sacked               -0.220067
pass_sacked_yds           -0

In [68]:
norm

,Humidity,OT,Off_DKP,Off_FDP,Off_SDP,Temperature,Total_DKP,Total_FDP,Total_SDP,Wind_Speed,...,rush_long,rush_scrambles,rush_td,rush_yac,rush_yds,rush_yds_before_contact,rush_yds_bonus,targets,two_point_conv,vis_score
0,-0.785871,-0.243981,-0.310963,-0.331249,-0.341430,0.916783,-0.310963,-0.331249,-0.341430,-0.932235,...,-0.396822,-0.185746,-0.167297,-0.297003,-0.361969,-0.343285,-0.083442,0.224834,-0.097973,1.117825
1,-0.785871,-0.243981,1.461757,1.307760,1.235715,0.916783,1.461757,1.307760,1.235715,-0.932235,...,-0.396822,-0.185746,-0.167297,-0.297003,-0.361969,-0.343285,-0.083442,3.207388,-0.097973,1.117825
2,0.925204,-0.243981,1.737131,1.919920,1.824770,-1.265029,1.737131,1.919920,1.824770,0.031514,...,1.115129,-0.185746,4.437348,3.767747,2.484124,1.198359,-0.083442,1.076992,-0.097973,-0.678666
3,1.210384,-0.243981,-0.655180,-0.627456,-0.626456,-1.209085,-0.655180,-0.627456,-0.626456,0.609763,...,-0.396822,-0.185746,-0.167297,-0.297003,-0.361969,-0.343285,-0.083442,-0.201245,-0.097973,1.329177
4,0.868168,-0.243981,-0.655180,-0.627456,-0.626456,-1.432861,-0.655180,-0.627456,-0.626456,0.224263,...,-0.396822,-0.185746,-0.167297,-0.297003,-0.361969,-0.343285,-0.083442,-0.201245,-0.097973,0.272417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,-0.785871,-0.243981,0.773322,0.419141,0.760672,0.916783,0.773322,0.419141,0.760672,-0.932235,...,-0.396822,-0.185746,-0.167297,-0.297003,-0.422524,-0.433970,-0.083442,-0.627324,-0.097973,1.329177
1152,-0.785871,-0.243981,-0.706813,-0.686697,-0.683461,0.916783,-0.706813,-0.686697,-0.683461,-0.932235,...,-0.850407,-0.185746,-0.167297,-0.297003,-0.543635,-0.615340,-0.083442,-0.627324,-0.097973,0.800797
1153,1.609634,-0.243981,-0.259330,-0.272008,-0.284425,-0.817478,-0.259330,-0.272008,-0.284425,2.344511,...,-0.396822,-0.185746,-0.167297,-0.297003,-0.361969,-0.343285,-0.083442,-0.201245,-0.097973,0.061065
1154,2.294065,-0.243981,-0.655180,-0.627456,-0.626456,1.364335,-0.655180,-0.627456,-0.626456,-0.161236,...,-0.396822,-0.185746,-0.167297,-0.297003,-0.361969,-0.343285,-0.083442,0.224834,-0.097973,-0.361638


In [69]:
norm.columns.get_indexer(['Humidity','index'])

array([ 0, 16], dtype=int64)

In [81]:
mn = weekly["index"].mean()
st = weekly["index"].std()
for i in X:
    normal_val = i[16]
    index_norm  = (normal_val * st) + mn
    print(round(index_norm))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
for 
mn = weekly["index"].mean()
st = weekly["string"].std()
index_norm  = (ypred * st) + mn


In [58]:
cnf_matrix = confusion_matrix(ytest,ypred)
print(cnf_matrix)
print(classification_report(ytest,ypred))

[[1026    0    0 ...    0    0    0]
 [   0    0    0 ...    0    0    0]
 [   0   25    0 ...    0    0    0]
 ...
 [   0    0    0 ...    0    0    0]
 [   0    0    0 ...    1    0    0]
 [   0    0    0 ...    1    0    0]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1026
         1.0       0.00      0.00      0.00         0
         2.0       0.00      0.00      0.00        26
         3.0       0.75      1.00      0.86         3
         4.0       1.00      0.80      0.89         5
         5.0       0.00      0.00      0.00         2
         6.0       0.00      0.00      0.00         0
         7.0       0.00      0.00      0.00         4
         8.0       1.00      0.50      0.67         2
         9.0       0.25      1.00      0.40         1
        10.0       0.00      0.00      0.00         0
        11.0       0.25      0.17      0.20         6
        12.0       0.00      0.00      0.00         2
        15.0   

C:\Users\carlo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\carlo\anaconda3\lib\site-packages\sklearn\metrics\_cla